In [8]:
#import social_interaction
#import code_interaction
#import buggy_commit
import pandas as pd
import numpy as np
import csv
import platform
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,KBinsDiscretizer
import statistics
from scipy.stats import pearsonr

In [9]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = 'Processed_data/'
else:
    _dir = 'Processed_data Data\\'
Files = [join(_dir, f) for f in listdir(_dir) if isfile(join(_dir, f))]
print(len(Files))
actual_file = 0
usable_files = 0
size_s = 0
size_m = 0
size_l = 0
result = []
size = []
total_df = pd.DataFrame()
for file in Files:
    try:
        _df = []
        actual_file += 1
        df = pd.read_pickle(file)
        buggy_commit_data_df = df.loc[0,'buggy_commit_data_df']
        commit_data_df = df.loc[0,'commit_data_df']
        commit_count = df.loc[0,'commit_count']
        sg_data_df = df.loc[0,'sg_data_df']
        cg_data_df = df.loc[0,'cg_data_df']
        buggy_commit_data_df = buggy_commit_data_df.rename(columns = {'count':'buggy_commit_count'})
        commit_data_df = commit_data_df.rename(columns = {'count':'commit_count'})
        sg_data_df = sg_data_df.rename(columns = {'count':'sg_node_count'})
        cg_data_df = cg_data_df.rename(columns = {'count':'cg_node_count'})
        #print("social",sg_data_df)
        #print("Code",cg_data_df)
        #print("commit",commit_data_df)
        #print("Buggy",buggy_commit_data_df)
        if commit_data_df.shape[0] > 30:
            size_l += 1
        elif commit_data_df.shape[0] > 15:
            size_m += 1
        elif commit_data_df.shape[0] > 8:
            size_s += 1
        if commit_data_df.shape[0] < 8:
            continue
        join_df = pd.merge(cg_data_df, sg_data_df, on='committer')
        join_df = pd.merge(join_df, buggy_commit_data_df, on='committer')
        join_df = pd.merge(join_df, commit_data_df, on='committer')
        join_df['buggy_percentage'] = join_df['buggy_commit_count']/join_df['commit_count']
        join_df_temp = join_df.drop(labels=['committer','buggy_percentage','commit_count','buggy_commit_count'],axis=1)
        scaler = MinMaxScaler()
        join_df_temp = scaler.fit_transform(join_df_temp)
        committer = join_df.committer.values.tolist()
        buggy_percentage = join_df.buggy_percentage.values.tolist()
        commit_count = join_df.commit_count.values.tolist()
        buggy_commit_count = join_df.buggy_commit_count.values.tolist()
        join_df = pd.DataFrame(join_df_temp,columns = ['cg_node_count','sg_node_count'])
        join_df['committer'] = committer
        join_df['buggy_percentage'] = buggy_percentage
        join_df['commit_count'] = commit_count
        join_df['buggy_commit_count'] = buggy_commit_count
        total_df = pd.concat([total_df,join_df])
        join_df.sort_values(by = ['cg_node_count','sg_node_count'],inplace=True,ascending=[False,True])
        _sum = np.sort(join_df.cg_node_count)
        #plt.plot(_sum)
        #plt.show()
        #print(join_df)
        #print(file.split('/',1)[1],join_df.iloc[0:int(join_df.shape[0]*0.2)].cg_node_count.sum(),join_df.cg_node_count.sum(),
        #     join_df.iloc[0:int(join_df.shape[0]*0.2)].cg_node_count.sum()/join_df.cg_node_count.sum(),join_df.shape[0])
        #for i in range(cg_data_df.shape[0]):
        #    buggy_commit_count = buggy_commit_data_df[buggy_commit_data_df['committer'] == cg_data_df.loc[i,'committer']]['count']
        result.append(join_df.iloc[0:int(join_df.shape[0]*0.2)].commit_count.sum()/join_df.commit_count.sum())
        size.append(join_df.shape[0])

    except:
        continue

12


In [10]:
# binning together
join_df_temp = total_df.drop(labels=['committer','buggy_percentage','commit_count','buggy_commit_count'],axis=1)
scaler = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')
join_df_temp = scaler.fit_transform(join_df_temp)
committer = total_df.committer.values.tolist()
buggy_percentage = total_df.buggy_percentage.values.tolist()
commit_count = total_df.commit_count.values.tolist()
buggy_commit_count = total_df.buggy_commit_count.values.tolist()
total_df = pd.DataFrame(join_df_temp,columns = ['cg_node_count','sg_node_count'])
total_df['committer'] = committer
total_df['buggy_percentage'] = buggy_percentage
total_df['commit_count'] = commit_count
total_df['buggy_commit_count'] = buggy_commit_count

In [11]:
pearsonr(total_df.sg_node_count.values.tolist(),total_df.cg_node_count.values.tolist())

(0.4012617253006687, 1.6788184580117035e-05)

In [12]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = 'Processed_data/'
else:
    _dir = 'Processed_data Data\\'
Files = [join(_dir, f) for f in listdir(_dir) if isfile(join(_dir, f))]
df_heros = pd.read_csv('hero_list.csv')
df_heros = df_heros[df_heros['heros_95'] == 'TRUE']
print(len(Files))
actual_file = 0
usable_files = 0
size_s = 0
size_m = 0
size_l = 0
result = []
size = []
total_df = pd.DataFrame()
for file in Files:
    try:
        _df = []
        actual_file += 1
        project = file.split('/')[1].split('_final')[0]
#         if project not in df_heros.repo_name.values.tolist():
#             print(project)
#             continue
        df = pd.read_pickle(file)
        buggy_commit_data_df = df.loc[0,'buggy_commit_data_df']
        commit_data_df = df.loc[0,'commit_data_df']
        commit_count = df.loc[0,'commit_count']
        sg_data_df = df.loc[0,'sg_data_df']
        cg_data_df = df.loc[0,'cg_data_df']
        buggy_commit_data_df = buggy_commit_data_df.rename(columns = {'count':'buggy_commit_count'})
        commit_data_df = commit_data_df.rename(columns = {'count':'commit_count'})
        sg_data_df = sg_data_df.rename(columns = {'count':'sg_node_count'})
        cg_data_df = cg_data_df.rename(columns = {'count':'cg_node_count'})
        #print("social",sg_data_df)
        #print("Code",cg_data_df)
        #print("commit",commit_data_df)
        #print("Buggy",buggy_commit_data_df)
        if commit_data_df.shape[0] > 30:
            size_l += 1
        elif commit_data_df.shape[0] > 15:
            size_m += 1
        elif commit_data_df.shape[0] > 8:
            size_s += 1
        if commit_data_df.shape[0] < 8:
            continue
        join_df = pd.merge(cg_data_df, sg_data_df, on='committer')
        join_df = pd.merge(join_df, buggy_commit_data_df, on='committer')
        join_df = pd.merge(join_df, commit_data_df, on='committer')
        join_df['buggy_percentage'] = join_df['buggy_commit_count']/join_df['commit_count']
        join_df.sort_values(by = ['cg_node_count','sg_node_count'],inplace=True,ascending=[False,True])
        join_df_temp = join_df.drop(labels=['committer','buggy_percentage','commit_count','buggy_commit_count'],axis=1)
        scaler = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
        join_df_temp = scaler.fit_transform(join_df_temp)
        committer = join_df.committer.values.tolist()
        buggy_percentage = join_df.buggy_percentage.values.tolist()
        commit_count = join_df.commit_count.values.tolist()
        buggy_commit_count = join_df.buggy_commit_count.values.tolist()
        join_df = pd.DataFrame(join_df_temp,columns = ['cg_node_count','sg_node_count'])
        join_df['committer'] = committer
        join_df['buggy_percentage'] = buggy_percentage
        join_df['commit_count'] = commit_count
        join_df['buggy_commit_count'] = buggy_commit_count
        total_df = pd.concat([total_df,join_df])
    except Exception as e:
        print(e,file)
        continue
total_df.reset_index(drop=True,inplace=True)

/Users/suvodeepmajumder/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


12


In [13]:
HH = []
HM = []
HL = []
MH = []
MM = []
ML = []
LH = []
LM = []
LL = []
for i in range(total_df.shape[0]):
    if total_df.iloc[i,0] == 2:
        if total_df.iloc[i,1] == 2:
            HH.append(total_df.iloc[i,3])
        elif total_df.iloc[i,1] == 1:
            HM.append(total_df.iloc[i,3])
        else:
            HL.append(total_df.iloc[i,3])
    elif total_df.iloc[i,0] == 1:
        if total_df.iloc[i,1] == 2:
            MH.append(total_df.iloc[i,3])
        elif total_df.iloc[i,1] == 1:
            MM.append(total_df.iloc[i,3])
        else:
            ML.append(total_df.iloc[i,3])
    else:
        if total_df.iloc[i,1] == 2:
            LH.append(total_df.iloc[i,3])
        elif total_df.iloc[i,1] == 1:
            LM.append(total_df.iloc[i,3])
        else:
            LL.append(total_df.iloc[i,3])

In [14]:
print('HH',statistics.median(HH),len(HH))
print('HM',statistics.median(HM),len(HM))
print('HL',statistics.median(HL),len(HL))
print('MH',statistics.median(MH),len(MH))
print('MM',statistics.median(MM),len(MM))
print('ML',statistics.median(ML),len(ML))
print('LH',statistics.median(LH),len(LH))
print('LM',statistics.median(LM),len(LM))
print('LL',statistics.median(LL),len(LL))

HH 0.34001806684733515 8
HM 0.3653846153846154 2
HL 0.6 5
MH 0.36363636363636365 1
MM 0.2780612244897959 2
ML 0.3055555555555556 3
LH 0.25925925925925924 7
LM 0.45918367346938777 5
LL 0.5 75


In [32]:
l = [HH,HM,HL,MH,MM,ML,LH,LM,LL]
names = ['HH','HM','HL','MH','MM','ML','LH','LM','LL']
with open('listfile.txt', 'w') as filehandle:
    for i in range(len(l)):
        filehandle.write('%s\n' % names[i])
        for listitem in l[i]:
            filehandle.write('%s ' % listitem)
        filehandle.write('\n')